rom doc2vec import Doc2VecModel,create_corpus,intents

In [ ]:
from utils import generate_embed_matrix
from utils import encode_pad_corpus
from keras.models import Sequential
from keras.layers.core import Dense , Flatten ,Dropout
from keras.layers import Embedding,LSTM,Bidirectional,BatchNormalization
#from keras.models import Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import  keras.callbacks as callbacks
#import ModelCheckpoint
# get the doc to vec model to retrieve the vocabulary size of the corpus
# generate the training data
# build the embedding model with keras library
# train the model on the docs
# save the model to retrieve the embedding matrix later

In [ ]:
       
        
def build_LSTM(vocab_size,output,embed_matrix):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size,output_dim=output, trainable=False,input_length=32,weights=[embed_matrix]))
    model.add(Bidirectional(LSTM(256, return_sequences=True, recurrent_dropout=0.1, dropout=0.1), 'concat'))
    model.add(Dropout(0.1))
    model.add(LSTM(256, return_sequences=False, recurrent_dropout=0.1, dropout=0.1))    
    model.add(Dropout(0.1))
    #model.add(Dense(512, activation='relu',kernel_regularizer ='l2'))
    model.add(Dense(512, activation='relu',kernel_regularizer ='l2'))

    #model.add(Flatten())
    model.add(Dense(11, activation='softmax'))
    
    return model

In [ ]:
X_data,Y_data,t,le=encode_pad_corpus()
print('X_data is: ', X_data)
embed_mat=generate_embed_matrix(t)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_data, Y_data, test_size = 0.3, 
                                                   shuffle = True, stratify=Y_data, random_state = 7)
vocab_size=len(t.word_counts) +1
print('Vocab size is: ', vocab_size)
lstm_model=build_LSTM(vocab_size,100,embed_mat)
lstm_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
lstm_model.summary()

nitializing checkpoint settings to view progress and save model

In [ ]:
filename = 'Models/Bi-LSTM_2.h5'

Learning rate scheduling<br>
This function keeps the initial learning rate for the first ten epochs  <br>
and decreases it exponentially after that.  

In [ ]:
def scheduler(epoch, learn_rate):
    if epoch < 20:
        return learn_rate
    else:
        return learn_rate * tf.math.exp(-0.1)

In [ ]:
sched_checkpoint = callbacks.LearningRateScheduler(scheduler)

This saves the best model<br>
checkpoint = callbacks.ModelCheckpoint(filename,  verbose=1, <br>
                             save_best_only=True, mode='min')

Fitting model with all the callbacks above

In [ ]:
hist = lstm_model.fit(X_train, y_train, epochs = 50, batch_size = 32, 
                 validation_data = (X_val, y_val), 
                 callbacks = [sched_checkpoint])

In [ ]:
lstm_model.save(filename)